# installation de azure

## base

In [ ]:
#!pip install --upgrade azureml-sdk

## extras

In [ ]:
#!pip install --upgrade azureml-sdk[explain,automl]

# setup

## azure

In [ ]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code GYWJSUQWK to authenticate.
Interactive authentication successfully completed.
Ready to use Azure ML 1.8.0 to work with jt-dp100


## drive
Pour pouvoir stocker le fichier config.json

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# workspace operations
[api azure : Workspace](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.workspace%28class%29?view=azure-ml-py#methods)

In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from pprint import pprint

ws.get_details()

for compute in ComputeTarget.list(ws):
    print('>>',compute.name, compute.type)
    if type(compute) is AmlCompute :
        pprint(vars(compute.get_status()))
    else:
        print(compute.get_status())
   

ModuleNotFoundError: ignored

# shtudown compute

[api azure : ComputeTarget](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.compute.computetarget?view=azure-ml-py)

[api azure : ComputeInstance](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.compute.computeinstance.computeinstance?view=azure-ml-py)

[api azure : AmlCompute](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.compute.amlcompute%28class%29?view=azure-ml-py)

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute, ComputeInstance
from azureml.core.compute_target import ComputeTargetException

compute_name = "vm-ds3-v2"

try:
    # Check for existing compute target
    compute = ComputeTarget(workspace=ws, name=compute_name)
    print('Found existing compute, use it.')
    
    if type(compute) is AmlCompute :
        compute.wait_for_completion(show_output=True, timeout_in_minutes=0)
    elif type(compute) is ComputeInstance:
        print('try to stop compute', compute_name)
        compute.stop()
        
    else :
        print(type(compute))

except ComputeTargetException:
    print('Not Found existing compute...',compute_name)



Found existing compute, use it.
try to stop compute vm-ds3-v2


# stop all compute instances from ws

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute, ComputeInstance
from azureml.core.compute_target import ComputeTargetException

for compute in ComputeTarget.list(ws):
    if type(compute) is ComputeInstance and compute.get_status().state == 'Running':
        print('try to stop compute', compute.name)
        compute.stop(show_output=True)

try to stop compute vm-ds3-v2


# start a compute instance

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute, ComputeInstance
from azureml.core.compute_target import ComputeTargetException

name = 'vm-ds3-v2'

for compute in ComputeTarget.list(ws):
    if type(compute) is ComputeInstance and compute.get_status().state == 'Stopped' and compute.name == name:
      print('try to start compute', compute.name)
      compute.start(show_output=True)


try to start compute vm-ds3-v2


# create a schedule task to stop all

In [ ]:
%%writefile stop_all.py
# Import libraries
import joblib

from azureml.core import Workspace, Model, Run
from azureml.core.compute import ComputeTarget, AmlCompute, ComputeInstance


# Get the experiment run context
run = Run.get_context()

ws = Workspace.from_config()

for compute in ComputeTarget.list(ws):
    if type(compute) is ComputeInstance and compute.get_status().state == 'Running':
        print('try to stop compute', compute.name)
        compute.stop(show_output=True)
        run.log('compute.name', 'try to stop compute')

run.complete()

Overwriting stop_all.py


In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "aml-cluster"

# Verify that cluster exists
try:
    pipeline_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If not, create it
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4,
                                                           idle_seconds_before_scaledown=1800)
    pipeline_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

pipeline_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [ ]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import RunConfiguration

# Create a Python environment for the experiment
stop_all_env = Environment("stop_all-pipeline-env")
stop_all_env.python.user_managed_dependencies = False # Let Azure ML manage dependencies
stop_all_env.docker.enabled = True # Use a docker container

# Create a set of package dependencies
stop_all_packages = CondaDependencies.create(conda_packages=[],
                                             pip_packages=['azureml-sdk'])

# Add the dependencies to the environment
stop_all_env.python.conda_dependencies = stop_all_packages

# Register the environment (just in case you want to use it again)
stop_all_env.register(workspace=ws)
registered_env = Environment.get(ws, 'stop_all-pipeline-env')

# Create a new runconfig object for the pipeline
pipeline_run_config = RunConfiguration()

# Use the compute you created above. 
pipeline_run_config.target = pipeline_cluster

# Assign the environment to the run configuration
pipeline_run_config.environment = registered_env

print ("Run configuration created.")

Run configuration created.


In [ ]:
from azureml.pipeline.core import PipelineData
from azureml.pipeline.steps import PythonScriptStep, EstimatorStep
from azureml.train.estimator import Estimator


estimator = Estimator(source_directory='.',
                        compute_target = pipeline_cluster,
                        environment_definition=pipeline_run_config.environment,
                        entry_script='stop_all.py')

stop_step = EstimatorStep(name = "Stop all",
                           estimator=estimator, 
                           estimator_entry_script_arguments=[],
                           compute_target = pipeline_cluster,
                           allow_reuse = True)


print("Pipeline step defined")

Pipeline step defined


In [ ]:
from azureml.core import Experiment
from azureml.pipeline.core import Pipeline
from azureml.widgets import RunDetails

# Construct the pipeline
pipeline_steps = [stop_step]
pipeline = Pipeline(workspace = ws, steps=pipeline_steps)
print("Pipeline is built.")

# Create an experiment and run the pipeline
experiment = Experiment(workspace = ws, name = 'stop_all-pipeline')
pipeline_run = experiment.submit(pipeline, regenerate_outputs=True)
print("Pipeline submitted for execution.")

pipeline_run.wait_for_completion()

Pipeline is built.
Created step Stop all [2e99dbf3][22047427-2265-4422-b0e6-6d6f0f09ae8f], (This step will run and generate new outputs)
Submitted PipelineRun 9659cfb2-4dfd-40cb-a7c4-a618b071c02c
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/stop_all-pipeline/runs/9659cfb2-4dfd-40cb-a7c4-a618b071c02c?wsid=/subscriptions/9114a63e-9210-4e32-97ca-b7d9e8ac403d/resourcegroups/jt-dp100-ressources/workspaces/jt-dp100
Pipeline submitted for execution.
PipelineRunId: 9659cfb2-4dfd-40cb-a7c4-a618b071c02c
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/stop_all-pipeline/runs/9659cfb2-4dfd-40cb-a7c4-a618b071c02c?wsid=/subscriptions/9114a63e-9210-4e32-97ca-b7d9e8ac403d/resourcegroups/jt-dp100-ressources/workspaces/jt-dp100
PipelineRun Status: NotStarted
PipelineRun Status: Running


StepRunId: 66c62a68-083a-455b-990a-0dc930fe12b3
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/stop_all-pipeline/runs/66c62a68-083a-455b-990a-0d

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "User program failed with ModuleNotFoundError: No module named 'joblib'",
        "detailsUri": "https://aka.ms/azureml-known-errors",
        "details": [],
        "debugInfo": {
            "type": "ModuleNotFoundError",
            "message": "No module named 'joblib'",
            "stackTrace": "  File \"/mnt/batch/tasks/shared/LS_root/jobs/jt-dp100/azureml/66c62a68-083a-455b-990a-0dc930fe12b3/mounts/workspaceblobstore/azureml/66c62a68-083a-455b-990a-0dc930fe12b3/azureml-setup/context_manager_injector.py\", line 148, in execute_with_context\n    runpy.run_path(sys.argv[0], globals(), run_name=\"__main__\")\n  File \"/azureml-envs/azureml_7ae22f719407a67a67557086d3c85ab0/lib/python3.6/runpy.py\", line 263, in run_path\n    pkg_name=pkg_name, script_name=fname)\n  File \"/azureml-envs/azureml_7ae22f719407a67a67557086d3c85ab0/lib/python3.6/runp